In [1]:
import pandas as pd
import time
# from skimage import io
# from skimage import filters
import skimage
from skimage import feature
from skimage import morphology
from skimage import measure
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy import ndimage as ndi
import numpy as np

start = time.time()

In [ ]:
pre_image = skimage.io.imread('../NPP001_Pre.tif')
pre_image = pre_image[0:int(pre_image.shape[0]/4),:]
fin_image = skimage.io.imread('../NPP001_fin.tif')
fin_image = fin_image[0:int(fin_image.shape[0]/4),:]

In [ ]:
# pre_x_min = 200
# pre_x_max = 3000-2000
# pre_y_min = 1500+1000
# pre_y_max = 4000-1000

# fin_x_min = 200
# fin_x_max = 3000-2000
# fin_y_min = 1500+1000
# fin_y_max = 4000-1000

pre_x_min = 0
pre_x_max = min(pre_image.shape[1], fin_image.shape[1])
pre_y_min = 0
pre_y_max = min(pre_image.shape[0], fin_image.shape[0])

fin_x_min = 0
fin_x_max = min(pre_image.shape[1], fin_image.shape[1])
fin_y_min = 0
fin_y_max = min(pre_image.shape[0], fin_image.shape[0])

bkg_subtract = np.subtract(pre_image[fin_y_min:fin_y_max, fin_x_min:fin_x_max].astype('int16'),
                           fin_image[pre_y_min:pre_y_max, pre_x_min:pre_x_max].astype('int16'))
bkg_subtract[bkg_subtract < 0] = 0
bkg_subtract = bkg_subtract.astype('uint8')

thresh = skimage.filters.threshold_otsu(bkg_subtract)
# print('Binarization threshold is', str(thresh))
binarized = bkg_subtract > thresh

In [ ]:
labeled_array, num_features = ndi.label(binarized)
region_info = measure.regionprops(labeled_array)
print('Found', str(num_features), 'features.')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(bkg_subtract)

for region in region_info:
    # take regions with reasonable areas
    if (region.area >= 2000 and region.area <=10000):
        # draw rectangle around segmented worms
        minr, minc, maxr, maxc = region.bbox
        rect = mpl.patches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

ax.set_ylim(1000, 4000)
ax.set_xlim(2000, 5000)
fig.savefig('../filtered_regions.png')

In [ ]:
# regions = morphology.watershed(binarized)
# print(str(int(time.time()-start)))

In [ ]:
# edges = feature.canny(bkg_subtract)

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
# axes[0].imshow(bkg_subtract[1000:4000, 2000:5000])
axes.imshow(binarized[1000:4000, 2000:5000])
fig.savefig('../binarized_section_of image.png')

In [ ]:
# plt.hist(pre_image.flatten(), bins=16)
# plt.show()

In [ ]:
# plt.hist(fin_image.flatten(), bins=16)
# plt.show()

In [ ]:
# plt.hist(bkg_subtract.flatten(), bins=16)
# plt.show()

In [ ]:
print('Notebook ran in', str(int(time.time()-start)), 'seconds.')